# 전처리

In [1]:
import pandas as pd
import os

rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [2]:
#3점 이상만 남김
ratings = ratings[ratings["ratings"]>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [3]:
# ratings 컬럼의 이름을 counts로 변경
ratings.rename(columns={"ratings":"counts"}, inplace=True)

In [4]:
#ratings 가  counts로 바뀐 것이 맞는지 확인
ratings['counts']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: counts, Length: 836478, dtype: int64

In [5]:
#타임스탬프는 영화를 시청한 날을 나타내므로 선호도와 관련이 없음=> 포함시키지 않음
use_columns = ["user_id", "movie_id", "counts"]
ratings = ratings[use_columns]
ratings.head()

,user_id,movie_id,counts
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5


ratings 전처리 끝

In [6]:
# 영화 제목을 보기 위해 데이터를 읽기
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
#타임스탬프는 영화를 시청한 날을 나타내므로 선호도와 관련이 없음=> 포함시키지 않음
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [7]:
#title, genre는 검색하기 쉽도록 하기 위해서 소문자로 변경
#영화 제목뒤에 붙은 년도는 제거하는 것이 맞을 수도 있으나
#리메이크의 가능성을 고려했을 때 삭제하지 않는 것이 바람직하다고 생각함

movies["title"] = movies["title"].str.lower()
movies["genre"] = movies["genre"].str.lower()

In [8]:
# | 표시를 알아보기 쉽게 ,로 변경
movies["genre"] = movies["genre"].str.replace("|", ",")

/tmp/ipykernel_312/2215937967.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  movies["genre"] = movies["genre"].str.replace("|", ",")


In [9]:
movies.head()

,movie_id,title,genre
0,1,toy story (1995),"animation,children's,comedy"
1,2,jumanji (1995),"adventure,children's,fantasy"
2,3,grumpier old men (1995),"comedy,romance"
3,4,waiting to exhale (1995),"comedy,drama"
4,5,father of the bride part ii (1995),comedy


movies 전처리 끝

# 분석

In [10]:
# user_id의 갯수와 movie_id의 갯수 확인
print(ratings["movie_id"].nunique())
print(ratings["user_id"].nunique())

3628
6039


In [11]:
ratings["movie_id"].isna().sum()

0

In [12]:
ratings["user_id"].isna().sum()

0

In [13]:
ratings["counts"].isna().sum()

0

ratings에서 결측치는 존재하지 않음

In [14]:
movies["movie_id"].isna().sum()

0

In [15]:
movies["title"].isna().sum()

0

In [16]:
movies["genre"].isna().sum()

0

movies에서도 결측치는 존재하지 않음

In [17]:
#ratings와 movies를 이어붙임 = 원활한 데이터 분석을 위해
data = pd.merge(movies, ratings)
data

,movie_id,title,genre,user_id,counts
0,1,toy story (1995),"animation,children's,comedy",1,5
1,1,toy story (1995),"animation,children's,comedy",6,4
2,1,toy story (1995),"animation,children's,comedy",8,4
3,1,toy story (1995),"animation,children's,comedy",9,5
4,1,toy story (1995),"animation,children's,comedy",10,5
...,...,...,...,...,...
836473,3952,"contender, the (2000)","drama,thriller",5682,3
836474,3952,"contender, the (2000)","drama,thriller",5812,4
836475,3952,"contender, the (2000)","drama,thriller",5831,3
836476,3952,"contender, the (2000)","drama,thriller",5837,4


In [18]:
#인기있는 영화순
movie_count = data.groupby("title")["user_id"].count()
movie_count.sort_values(ascending=False).head(30)

title
american beauty (1999)                                   3211
star wars: episode iv - a new hope (1977)                2910
star wars: episode v - the empire strikes back (1980)    2885
star wars: episode vi - return of the jedi (1983)        2716
saving private ryan (1998)                               2561
terminator 2: judgment day (1991)                        2509
silence of the lambs, the (1991)                         2498
raiders of the lost ark (1981)                           2473
back to the future (1985)                                2460
matrix, the (1999)                                       2434
jurassic park (1993)                                     2413
sixth sense, the (1999)                                  2385
fargo (1996)                                             2371
braveheart (1995)                                        2314
men in black (1997)                                      2297
schindler's list (1993)                                  2257
pr

In [19]:
# 유저별 몇 편의 영화를 평가했는지
user_count = data.groupby("user_id")["title"].count()
user_count.describe()

count    6039.000000
mean      138.512668
std       156.241599
min         1.000000
25%        38.000000
50%        81.000000
75%       177.000000
max      1968.000000
Name: title, dtype: float64

# 선호영화 추가

In [20]:
#영화목록 확인 - series파일을 바로 확인할 수 없기때문에 파일타입 확인
fav = movies["title"]
type(fav)

pandas.core.series.Series

In [21]:
#영화목록 확인 = tolist()를 사용하여 list 형태로 변환
bb = fav.tolist()
bb

['toy story (1995)',
 'jumanji (1995)',
 'grumpier old men (1995)',
 'waiting to exhale (1995)',
 'father of the bride part ii (1995)',
 'heat (1995)',
 'sabrina (1995)',
 'tom and huck (1995)',
 'sudden death (1995)',
 'goldeneye (1995)',
 'american president, the (1995)',
 'dracula: dead and loving it (1995)',
 'balto (1995)',
 'nixon (1995)',
 'cutthroat island (1995)',
 'casino (1995)',
 'sense and sensibility (1995)',
 'four rooms (1995)',
 'ace ventura: when nature calls (1995)',
 'money train (1995)',
 'get shorty (1995)',
 'copycat (1995)',
 'assassins (1995)',
 'powder (1995)',
 'leaving las vegas (1995)',
 'othello (1995)',
 'now and then (1995)',
 'persuasion (1995)',
 'city of lost children, the (1995)',
 'shanghai triad (yao a yao yao dao waipo qiao) (1995)',
 'dangerous minds (1995)',
 'twelve monkeys (1995)',
 'wings of courage (1995)',
 'babe (1995)',
 'carrington (1995)',
 'dead man walking (1995)',
 'across the sea of time (1995)',
 'it takes two (1995)',
 'clueless (

In [22]:
#내가 선호하는 5가지 영화
my_favorite = ["Jumanji (1995)", "men in black (1997)", "terminator 2: judgment day (1991)", "ghostbusters (1984)", "wallace & gromit: the best of aardman animation (1996)"]

# yyttt라는 user_id가 위 영화를 5점씩 주었다고 가정 # 이 정보를 바탕으로 추천받기위해
my_playlist = pd.DataFrame({'user_id': ['yyttt']*5, 'title': my_favorite, 'counts':[5]*5})

# user_id에 'yyttt'라는 데이터가 없다면
# 위에 임의로 만든 my_favorite 데이터를 추가
if not data.isin({"user_id":["yyttt"]})["user_id"].any():
    data = data.append(my_playlist)
    
data.tail(10)

,movie_id,title,genre,user_id,counts
836473,3952.0,"contender, the (2000)","drama,thriller",5682,3
836474,3952.0,"contender, the (2000)","drama,thriller",5812,4
836475,3952.0,"contender, the (2000)","drama,thriller",5831,3
836476,3952.0,"contender, the (2000)","drama,thriller",5837,4
836477,3952.0,"contender, the (2000)","drama,thriller",5998,4
0,NaN,Jumanji (1995),NaN,yyttt,5
1,NaN,men in black (1997),NaN,yyttt,5
2,NaN,terminator 2: judgment day (1991),NaN,yyttt,5
3,NaN,ghostbusters (1984),NaN,yyttt,5
4,NaN,wallace & gromit: the best of aardman animatio...,NaN,yyttt,5


In [23]:
#movie_id, genre는 필요없는 부분이므로 삭제 = 이 부분만 결측치가 생기게되므로
data = data.drop(columns=["movie_id", "genre"])
data.tail(10)

,title,user_id,counts
836473,"contender, the (2000)",5682,3
836474,"contender, the (2000)",5812,4
836475,"contender, the (2000)",5831,3
836476,"contender, the (2000)",5837,4
836477,"contender, the (2000)",5998,4
0,Jumanji (1995),yyttt,5
1,men in black (1997),yyttt,5
2,terminator 2: judgment day (1991),yyttt,5
3,ghostbusters (1984),yyttt,5
4,wallace & gromit: the best of aardman animatio...,yyttt,5


In [24]:
# 고유한 유저, title을 찾아내는 코드
user_unique = data["user_id"].unique()
title_unique = data["title"].unique()

# 유저, 아티스트 indexing 하는 코드
user_to_idx = {v:k for k,v in enumerate(user_unique)}
title_to_idx = {v:k for k,v in enumerate(title_unique)}

In [25]:
#유저 수 확인 - yyttt를 추가한 이후
data["user_id"].nunique()

6040

In [26]:
# 인덱싱이 잘 되었는지 확인
print(user_to_idx['yyttt'])    # 6040명의 유저 중 마지막으로 추가된 유저이니 6039가 나와야 함
print(title_to_idx["terminator 2: judgment day (1991)"])

6039
569


In [27]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구하기
# 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거 

temp_user_data = data['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(data):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    data['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체
else:
    print('user_id column indexing Fail!!')

# temp_title_data을 통해 title 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_title_data = data['title'].map(title_to_idx.get).dropna()
if len(temp_title_data) == len(data):
    print('title column indexing OK!!')
    data['title'] = temp_title_data
else:
    print('title column indexing Fail!!')

data

user_id column indexing OK!!
title column indexing OK!!


,title,user_id,counts
0,0,0,5
1,0,1,4
2,0,2,4
3,0,3,5
4,0,4,5
...,...,...,...
0,3628,6039,5
1,1419,6039,5
2,569,6039,5
3,2462,6039,5


# CSR matrix 만들기

In [28]:
from scipy.sparse import csr_matrix

num_user = data['user_id'].nunique()
num_movie = data['title'].nunique()

csr_data = csr_matrix((data.counts, (data.user_id, data.title)), shape= (num_user, num_movie))
csr_data

<6040x3629 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

## als_model = AlternatingLeastSquares 모델을 구성

In [29]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하는 부분
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [37]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=400, regularization=0.01, use_gpu=False, iterations=200, dtype=np.float32)

In [38]:
# als 모델은 input으로 아래 csr_data_transpose를 값으로 넣음(item X user 꼴의 matrix를 받기 때문에 Transpose해주어야함)
csr_data_transpose = csr_data.T
csr_data_transpose

<3629x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [39]:
#훈련시작
als_model.fit(csr_data_transpose)

  0%|          | 0/200 [00:00<?, ?it/s]

# 선호 영화와 사용자에 대한 벡터값 확인

In [40]:
yyttt, ghostbusters = user_to_idx["yyttt"], title_to_idx["ghostbusters (1984)"]
yyttt_vector, ghostbusters_vector = als_model.user_factors[yyttt], als_model.item_factors[ghostbusters]

In [41]:
yyttt_vector

array([-0.13788937,  0.06698648, -0.14684267, -0.06558671,  0.20143026,
        0.02065133, -0.00952304,  0.15658239,  0.16335928,  0.04353756,
        0.0267273 ,  0.17942409,  0.01020977,  0.02883414, -0.05711428,
       -0.05173185,  0.07056852, -0.23712404, -0.04946279,  0.2489242 ,
       -0.01204241, -0.01435497,  0.05490415, -0.2666654 ,  0.20609634,
       -0.17848678, -0.08921748,  0.05321425, -0.02855433, -0.09635437,
        0.0271368 , -0.0126365 , -0.06977362, -0.03724172,  0.03090444,
       -0.12318131, -0.02751229,  0.14211492,  0.02363663,  0.08496514,
        0.14516295,  0.10152077, -0.22610433, -0.14280748,  0.25495157,
        0.00568701,  0.09132111, -0.0201713 ,  0.09544934, -0.25214726,
       -0.02692058,  0.20472382, -0.0266011 ,  0.14075987, -0.24038938,
       -0.24510309,  0.06044336,  0.15102698,  0.23927033, -0.14390385,
       -0.06010812, -0.1096753 ,  0.01715902,  0.00169036,  0.0743733 ,
       -0.07644209,  0.08441053, -0.15587522, -0.04017431,  0.01

In [42]:
ghostbusters_vector

array([-2.04004347e-02,  2.13642549e-02, -3.13032418e-02,  3.93438078e-02,
        5.99058829e-02,  3.40788178e-02,  3.23829800e-02,  2.04783585e-02,
        7.15262741e-02, -6.04979768e-02,  1.32695092e-02,  8.52662101e-02,
        2.77796350e-02, -4.20868322e-02,  3.53492089e-02,  4.95809466e-02,
        8.64054188e-02, -4.95036542e-02, -1.22308517e-02,  8.99670571e-02,
        1.08584519e-02,  7.14549236e-03,  2.71570720e-02, -4.98143733e-02,
        4.00847122e-02, -4.79938798e-02,  3.72149609e-03,  8.23739450e-03,
       -6.35760128e-02,  4.09429008e-03,  3.24848965e-02, -2.93080471e-02,
        4.16787453e-02, -4.89512313e-05,  7.23015470e-03, -6.84847012e-02,
        3.38022076e-02, -3.72841116e-03,  3.29754055e-02,  3.07254959e-03,
        1.98292285e-02, -3.30325239e-03, -1.02258116e-01, -1.15164109e-01,
        3.68173644e-02,  2.77359672e-02, -3.51726897e-02,  8.83114990e-03,
        6.09842986e-02,  2.71509513e-02, -3.13868299e-02,  3.32037248e-02,
       -3.92520502e-02,  

# 모델이 예측한 선호도

In [43]:
np.dot(yyttt_vector, ghostbusters_vector)

0.86217344

factors=100, iterations=200 시 예측선호도 0.39

factors=400, iterations=200 시 예측선호도 0.86

In [46]:
#매트릭스에 대한 선호도 예측
matrixx = title_to_idx['matrix, the (1999)']
matrixx_vector = als_model.item_factors[matrixx]
np.dot(yyttt_vector, matrixx_vector)

0.12776265

# 비슷한 영화 추천받기

In [50]:
idx_to_title = {v:k for k,v in title_to_idx.items()}

def get_similar_title(title_name: str):
    title_id = title_to_idx[title_name]
    similar_title = als_model.similar_items(title_id)
    similar_title = [idx_to_title[i[0]] for i in similar_title]
    return similar_title

In [51]:
get_similar_title("ghostbusters (1984)")

['ghostbusters (1984)',
 'Jumanji (1995)',
 'ghostbusters ii (1989)',
 'gremlins (1984)',
 'random hearts (1999)',
 'for ever mozart (1996)',
 "general's daughter, the (1999)",
 'make them die slowly (cannibal ferox) (1980)',
 "ferris bueller's day off (1986)",
 'out of africa (1985)']

In [52]:
get_similar_title("Jumanji (1995)")

['Jumanji (1995)',
 'wallace & gromit: the best of aardman animation (1996)',
 'ghostbusters (1984)',
 'men in black (1997)',
 'terminator 2: judgment day (1991)',
 'bushwhacked (1995)',
 'hamlet (1964)',
 'better than chocolate (1999)',
 'monument ave. (1998)',
 'loser (2000)']

In [53]:
get_similar_title("men in black (1997)")

['men in black (1997)',
 'Jumanji (1995)',
 'jurassic park (1993)',
 'digimon: the movie (2000)',
 'my life so far (1999)',
 'terminator 2: judgment day (1991)',
 'man of no importance, a (1994)',
 'total recall (1990)',
 'bewegte mann, der (1994)',
 'female perversions (1996)']

In [54]:
get_similar_title("terminator 2: judgment day (1991)")

['terminator 2: judgment day (1991)',
 'Jumanji (1995)',
 'terminator, the (1984)',
 'total recall (1990)',
 'matrix, the (1999)',
 'city of the living dead (paura nella città dei morti viventi) (1980)',
 'men in black (1997)',
 'fugitive, the (1993)',
 'jurassic park (1993)',
 'man from down under, the (1943)']

In [55]:
get_similar_title("wallace & gromit: the best of aardman animation (1996)")

['wallace & gromit: the best of aardman animation (1996)',
 'Jumanji (1995)',
 'wrong trousers, the (1993)',
 'close shave, a (1995)',
 'grand day out, a (1992)',
 'creature comforts (1990)',
 'kid, the (1921)',
 'haunted world of edward d. wood jr., the (1995)',
 'loser (2000)',
 'nothing to lose (1994)']

# 가장 좋아할만한 영화 추천받기

In [56]:
#유저에게 추천하기
user = user_to_idx['yyttt']
# recommend에서는 user*item CSR Matrix를 받음
title_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
title_recommended

[(1041, 0.28194997),
 (693, 0.24868338),
 (1107, 0.21415704),
 (1122, 0.19322468),
 (2657, 0.1713377),
 (3546, 0.14553864),
 (2507, 0.13397174),
 (2325, 0.12776268),
 (462, 0.12267098),
 (2542, 0.12227506),
 (2463, 0.1143137),
 (853, 0.112234466),
 (1904, 0.11092536),
 (2741, 0.10852772),
 (1131, 0.10569991),
 (106, 0.10345056),
 (2050, 0.09994529),
 (2660, 0.09758504),
 (1120, 0.09574095),
 (2536, 0.09554158)]

영화 이름이 인덱스로 나오기떄문에 index_to_title로 변환이 필요

In [58]:
[idx_to_title[i[0]] for i in title_recommended]

['wrong trousers, the (1993)',
 'close shave, a (1995)',
 'grand day out, a (1992)',
 'terminator, the (1984)',
 'total recall (1990)',
 'naked gun: from the files of police squad!, the (1988)',
 'sixth sense, the (1999)',
 'matrix, the (1999)',
 'jurassic park (1993)',
 'big (1988)',
 'ghostbusters ii (1989)',
 'father of the bride (1950)',
 'addams family, the (1991)',
 'bone collector, the (1999)',
 'nikita (la femme nikita) (1990)',
 'braveheart (1995)',
 'thing, the (1982)',
 'year of living dangerously (1982)',
 'seventh seal, the (sjunde inseglet, det) (1957)',
 'airplane! (1980)']

In [59]:
wrong_trousers = title_to_idx["wrong trousers, the (1993)"]
explain = als_model.explain(user, csr_data, itemid=wrong_trousers)

In [60]:
[(idx_to_title[i[0]], i[1]) for i in explain[1]]

[('wallace & gromit: the best of aardman animation (1996)',
  0.2744289613893202),
 ('terminator 2: judgment day (1991)', 0.015065644789715435),
 ('Jumanji (1995)', 0.00045642982662666773),
 ('ghostbusters (1984)', 7.545432057014592e-05),
 ('men in black (1997)', -0.00857888431498738)]

# 회고

* 오늘따라 하고싶은 것은 많았는데, 기능만 찾다가 시간을 상당 수 쏟은 듯 합니다.
* 셀 2개를 합치기 위해서 concat을 사용했으나 이상한 결과값이 나온 탓에 비슷하면서 다른 기능인 merge를 알게 되었습니다.
* 뿐만 아니라 del 을 사용하여 columns를 제거하려 했으나 잘 되지 않은 탓에 dropna를 사용하여 columns를 제거했습니다.
* groupby를 사용할 경우 보통 1가지 값을 사용하는데, 이 경우에는 2가지 이상을 사용하여서 알아내느라 시간을 보냈습니다. 위에서 사용한 movie_count = data.groupby("title")["user_id"].count() 이 방법도 괜찮은 듯 싶지만, movie_count = data.groupby("title").sum()을 하는 것도 괜찮지 않을까 라는 생각이 듭니다.>>> 써보니 연산량이 너무 많아서 돌리는데 아주 오랜시간이 돌려서 종료했습니다. 단순히 count를 쓰는 것이 좋습니다
* 예전부터 느꼈지만 판다스를 이용한 슬라이싱부분에서 특히나 약한 모습을 보이고 있습니다. 캐글필사 외에도 다른 방법을 통해서 보완할 수 있어야겠습니다.